<a href="https://colab.research.google.com/github/JosephFalconio/Joseph-Falconio_dissertation/blob/main/clean3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Step 1: Load data
age_df = pd.read_csv('/content/travel_method_2011_spec.csv')
lookup = pd.read_csv('lookup_2011_2021_leeds.csv')

# Step 2: Keep relevant columns
lookup = lookup[['LSOA11CD', 'LSOA21CD', 'ChangeType']]

# Step 3: Merge age data with lookup
merged = pd.merge(lookup, age_df, on='LSOA11CD', how='left')

# Step 4: Handle 'unchanged'
unchanged = merged[merged['ChangeType'] == 'U'].drop(columns=['LSOA11CD', 'ChangeType'])

# Step 5: Handle 'split' — replicate values
split = merged[merged['ChangeType'] == 'S'].drop(columns=['LSOA11CD', 'ChangeType'])

# Step 6: Handle 'merged' — sum up each column by new LSOA
merged_group = merged[merged['ChangeType'] == 'M'].drop(columns=['LSOA11CD', 'ChangeType']).groupby('LSOA21CD').sum(numeric_only=True).reset_index()

# Step 7: Handle 'irregular' — treat like merged
irregular_group = merged[merged['ChangeType'] == 'X'].drop(columns=['LSOA11CD', 'ChangeType']).groupby('LSOA21CD').sum(numeric_only=True).reset_index()

# Step 8: Combine all results
combined = pd.concat([unchanged, split, merged_group, irregular_group], ignore_index=True)

# Step 9: Clean up duplicates just in case
combined = combined.groupby('LSOA21CD').sum(numeric_only=True).reset_index()

# Step 10: Save to CSV
combined.to_csv('travel_method_2011_converted_to_2021_LSOA.csv', index=False)

print(combined.head())

    LSOA21CD  All categories: Method of travel to work    WFH  \
0  E01011264                                     950.0   59.0   
1  E01011265                                    1114.0   79.0   
2  E01011266                                    1574.0  145.0   
3  E01011267                                    1022.0   62.0   
4  E01011268                                     983.0   38.0   

   Underground, metro, light rail, tram  Train  Bus, minibus or coach  Taxi  \
0                                   1.0   30.0                   40.0   4.0   
1                                   1.0  167.0                   25.0   2.0   
2                                   3.0  163.0                   20.0   4.0   
3                                   0.0   85.0                   34.0   2.0   
4                                   0.0   29.0                   43.0   3.0   

   Motorcycle, scooter or moped  Driving a car or van  \
0                           5.0                 379.0   
1                   